In [1]:
import numpy as np
import pandas as pd
from timeit import default_timer as timer
from numba import jit, njit, vectorize, cfunc
from scipy import sparse
from scipy.sparse.linalg import spsolve
import networkx as nx
from scipy.sparse.csgraph import reverse_cuthill_mckee


In [2]:
#####################
# friction functions#
#####################

@njit(parallel=True)
def f_swamee(epsD,Re):
    y1=5.74/np.power(Re,0.9)
    y2=epsD/3.7+y1
    c1=-2/np.log(10)
    y3=c1*np.log(y2)
    f=1/(y3*y3)

    y3=c1*np.log(y2)
    fp=(2*0.9*c1)*y1*f/(y2*y3*Re)
    
    return [f,fp]

@njit(parallel=True)
def f_dunlop(epsD,Re):
    aa=-1.5634601348517065795
    #aa=-2*0.9*2/np.log(10)
    ab=0.00328895476345399058690
    #ab=5.74/np.power(4000,0.9)
    c2=-2/np.log(10)
    #c2=-0.8685889638065035
    
    ac=aa*ab
    
    y2=epsD/3.7+ab
    y3=c2*np.log(y2)
    
    fa=1/(y3*y3)
    fb=(2+ac/(y2*y3))*fa
    
    x1=7*fa-fb
    x2=0.128-17*fa+2.5*fb
    x3=-0.128+13*fa-(fb+fb)
    x4=0.032-3*fa+0.5*fb
    
    R=Re/2000
      
    f=x1+R*(x2+R*(x3+R*x4))
    fp=x2+R*(2*x3+3*R*x4)
    fp=fp/2000
    
    return [f,fp]

@njit(parallel=True)
def f_lam(epsD,Re):
    return [64/Re -64/(Re*Re)]


@njit
def smoke1(F, b, i, j,Np):
    for k in range(Np):
        F[i[k]]+=b[k]
        F[j[k]]-=b[k]

#####################
#  other function   #
#####################
def file_read(name_,name_n):
    node = pd.read_csv(name_[name_n]+'\\junction.txt',sep='\t')
    pipe = pd.read_csv(name_[name_n]+'\\pipe.txt',sep='\t')
    reservoir =   pd.read_csv(name_[name_n]+'\\reservoir.txt',sep='\t')
    return [node, pipe, reservoir]

In [3]:
np.show_config()

blas_mkl_info:
  NOT AVAILABLE
blis_info:
  NOT AVAILABLE
openblas_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
blas_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_info']
    libraries = ['openblas_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_mkl_info:
  NOT AVAILABLE
openblas_lapack_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]
lapack_opt_info:
    library_dirs = ['D:\\a\\1\\s\\numpy\\build\\openblas_lapack_info']
    libraries = ['openblas_lapack_info']
    language = f77
    define_macros = [('HAVE_CBLAS', None)]


In [4]:
## file read ##
name_n=3


In [5]:
## file read ##
###############
start = timer()
###############
name_=['example4','example5','bin','rome'];
demand_multip=np.array([1,1,0.45,1])

a=file_read(name_,name_n)

node=a[0];
pipe=a[1];
reservoir=a[2];

print('file read, time=',timer() - start,'s')
print('\n')




## initialize##
###############
start = timer()
###############

node_id=np.array(node.ID)-1
elev=np.array(node.Elev) #m
demand=np.array(node.Demand/1000)# %m^3/s
demand=demand*demand_multip[name_n]
pipe_id=np.array(pipe.ID)-1
pipe_node=np.array([np.array(pipe.Node1)-1 , np.array(pipe.Node2)-1])
len_=np.array(pipe.Length) #m
diam=np.array(pipe.Diameter/1000) #m
rough=np.array(pipe.Roughness/1000) #m
rsrv_=[np.array(reservoir.ID) ,np.array(reservoir.Head)]   # head(m)
rsrv_1=rsrv_[1]

Nr=len(rsrv_)
Np=len(pipe_id)
Nj=len(node_id)
N=Nj-Nr

V=0.3048*np.ones(Np)

q=np.ones(Np)
c1=np.pi/4
q=c1*diam*diam*V # m^3/s


nu=0.000001  #m^2/s

h=np.ones(Nj)
Re=np.zeros(Np)
epsD=np.zeros(Np)

epsD=rough/diam

print('initializing variables, time=',timer() - start,'s')

file read, time= 0.1780381000000002 s


initializing variables, time= 0.07675220000000005 s


In [6]:
################################
#this must be repeated several times

In [7]:
n_iter=30
for jj in range(n_iter):

    print('\n iter',jj)

    ###############
    ## Reynolds  ##
    ###############
    start = timer()
    ###############
    Re=(4/(np.pi*nu))*abs(q)/diam;

    oo=np.ones(len(Re))

    Re_turb=np.maximum(Re,4000)
    Re_trans=np.minimum(np.maximum(Re,2000),4000)
    Re_lam=np.minimum(Re,2000)

    a_turb=1*(Re>=4000)
    a_trans=(Re<4000)*(Re>2000)*1
    a_lam=1*(Re<=2000)

    f=np.zeros(Np)
    fp=np.zeros(Np)

    print('Calculate Re, time=',timer() - start,'s')




    #print('\n')



    ###############
    ## friction  ##
    ###############

    #print('turbulent f')
    #%timeit f_swamee(epsD,Re)
    #print('transition f')
    #%timeit f_dunlop(epsD,Re)
    #print('laminar f')
    #%timeit f_lam(epsD,Re)


    ###############
    start = timer()
    ###############
    a1=f_swamee(epsD,Re)
    print('turbulent f, time=',timer() - start,'s')

    ###############
    start = timer()
    ###############
    a2=f_dunlop(epsD,Re)
    print('transition f, time=',timer() - start,'s')

    ###############
    start = timer()
    ###############
    a3=f_lam(epsD,Re)
    print('laminar f, time=',timer() - start,'s')

    ###############
    start = timer()
    ###############
    a=a_turb*a1+a_trans*a2+a_lam*a3
    f=a[0]
    fp=a[1]
    print('Calculate f & fp, time=',timer() - start,'s')





    #print('\n')


    ###############
    ## hL & g    ##
    ###############

    ###############
    start = timer()
    ###############

    #c1=0.0826268572006832
    c1=8/(np.power(np.pi,2)*9.81)
    r_turb=c1*len_/(diam*diam*diam*diam*diam)
    #r_turb=c1*len_/np.power(diam,5)
    hL=r_turb*f*q*abs(q)

    g=r_turb*abs(q)*(2*f+Re*fp)

    print('calculate h_L & g, time=',timer() - start,'s')



    #print('\n')

    ###############
    ##     A     ##
    ###############

    ###############
    start = timer()
    ###############

    nnn=(Nr+1)+(Nj+1)
    A=sparse.csr_matrix((nnn, nnn))

    a=1/g
    i=pipe_node[0]
    j=pipe_node[1]

    A=A+sparse.csr_matrix((-a, (i, j)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix((-a, (j, i)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix(( a, (i, i)), shape=(nnn, nnn))
    A=A+sparse.csr_matrix(( a, (j, j)), shape=(nnn, nnn))

    B=A
    A=sparse.csr_matrix((Nj+1, Nj+1))

    A=B[0:Nj,0:Nj]
    print('matrix A, time=',timer() - start,'s')


    #print('\n')


    ###############
    ##     F     ##
    ###############

    ###############
    start = timer()
    ###############

    F=np.zeros(nnn)


    b=-q+hL/g




    ###############
    #start = timer()
    ###############    

    
    #del i , j

    #F=np.bincount(i,weights=b,minlength=nnn)-np.bincount(j,weights=b,minlength=nnn)
    smoke1(F, np.asarray(b), np.asarray(i), np.asarray(j),Np)
    stor_ind_i=np.where(i>=Nj)[0]
    stor_ind_j=np.where(j>=Nj)[0]
    for kk in range(len(stor_ind_i)):
        k=stor_ind_i[kk]
        F[j[k]]+=rsrv_1[i[k]-Nj]/g[k]
    for kk in range(len(stor_ind_j)):
        k=stor_ind_j[kk]
        F[i[k]]+=rsrv_1[j[k]-Nj]/g[k]  
    
    #original slow code
    #for k in range(Np):
        #i=pipe_node[0][k]
        #j=pipe_node[1][k]
        #i1=i[k]
        #j1=j[k]
        #F[i1]=F[i1]+b[k]
        #F[j1]=F[j1]-b[k]
        #if i1>=Nj:
            #F[j1]=F[j1]+rsrv_[1][i1-Nj]/g[k];
        #elif j1>=Nj:
            #F[i1]=F[i1]+rsrv_[1][j1-Nj]/g[k];

    FF=F
    F=np.zeros(Nj)
    F[0:Nj]=FF[0:Nj]
    F=F-demand

    print('vecF, time=',timer() - start,'s')

    #print('\n')


    
    
    
    ###############
    ##   solve   ##
    ###############

    ###############
    start = timer()
    ###############

    h=sparse.linalg.spsolve(A,F)
    print('solve system, time=',timer() - start,'s')
    #%timeit sparse.linalg.spsolve(A,F)



    #print('\n')
    
    ###############
    ## update q  ##
    ###############

    ###############
    start = timer()
    ###############

    dq=np.zeros(Np);
    #dq=hL/g
    hh=np.append(h,rsrv_[1])  #it's easier to do this than handle indices
    
    dq=(hL-hh[i]+hh[j])/g;
    #original slow loop
    #for k in range(Np):
        #i1=i[k]
        #j1=j[k]
        #dq[k]=(hL[k]-hh[i[k]]+hh[j[k]])/g[k];
        #if i1<Nj and j1<Nj:
            #dq[k]=(hL[k]-h[i1]+h[j1])/g[k];
        #elif i1>=Nj:
            #dq[k]=(hL[k]-rsrv_[1][i1-Nj]+h[j1])/g[k]
        #elif j1>=Nj:
            #dq[k]=(hL[k]-h[i1]+rsrv_[1][j1-Nj])/g[k]
    
    q=q-dq

    print('updating q, time=',timer() - start,'s')
    print(q*1000)


 iter 0
Calculate Re, time= 0.006973099999999732 s
turbulent f, time= 0.9143002000000005 s
transition f, time= 1.1960408999999999 s
laminar f, time= 0.5421652999999997 s
Calculate f & fp, time= 0.01408310000000057 s
calculate h_L & g, time= 0.015927900000000328 s
matrix A, time= 0.06821570000000055 s
vecF, time= 0.1779326000000001 s
solve system, time= 0.21674619999999933 s
updating q, time= 0.006026600000000215 s
[-7.91351255  0.2232     -8.16315355 ...  0.073334    0.058981
  0.021212  ]

 iter 1
Calculate Re, time= 0.011244599999999494 s
turbulent f, time= 0.0030384000000003297 s
transition f, time= 0.0016398000000004131 s
laminar f, time= 0.0006788999999995937 s
Calculate f & fp, time= 0.01538459999999997 s
calculate h_L & g, time= 0.015965299999999516 s
matrix A, time= 0.07443059999999946 s
vecF, time= 0.005288900000000041 s
solve system, time= 0.22676989999999986 s
updating q, time= 0.005639300000000347 s
[-7.47356006  0.2232     -7.72320106 ...  0.073334    0.058981
  0.021212 

solve system, time= 0.22975759999999923 s
updating q, time= 0.005938199999999227 s
[-7.1889799  0.2232    -7.4386209 ...  0.073334   0.058981   0.021212 ]

 iter 17
Calculate Re, time= 0.009316000000000102 s
turbulent f, time= 0.00382040000000039 s
transition f, time= 0.001629300000001166 s
laminar f, time= 0.0006891000000006642 s
Calculate f & fp, time= 0.015552399999998912 s
calculate h_L & g, time= 0.01822630000000025 s
matrix A, time= 0.07362270000000137 s
vecF, time= 0.0052501999999989835 s
solve system, time= 0.23884499999999953 s
updating q, time= 0.00913689999999967 s
[-7.1890334  0.2232    -7.4386744 ...  0.073334   0.058981   0.021212 ]

 iter 18
Calculate Re, time= 0.01115579999999916 s
turbulent f, time= 0.018114199999999414 s
transition f, time= 0.004501600000001105 s
laminar f, time= 0.0008943999999999619 s
Calculate f & fp, time= 0.017333800000001176 s
calculate h_L & g, time= 0.016105500000000106 s
matrix A, time= 0.0836238999999992 s
vecF, time= 0.005543900000001045 s


In [8]:
q*1000
#expected result
#array([-7.18357863,  0.2232    , -7.43321963, ...,  0.073334  ,        0.058981  ,  0.021212  ])

array([-7.1833065,  0.2232   , -7.4329475, ...,  0.073334 ,  0.058981 ,
        0.021212 ])

In [9]:
perm=reverse_cuthill_mckee(A)
perm
def permute_sparse_matrix(M, row_order=None, col_order=None):
    """
    Reorders the rows and/or columns in a scipy sparse matrix to the specified order.
    """
    if row_order is None and col_order is None:
        return M
    
    new_M = M
    if row_order is not None:
        I = sparse.eye(M.shape[0]).tocoo()
        I.row = I.row[row_order]
        new_M = I.dot(new_M)
    if col_order is not None:
        I = sparse.eye(M.shape[1]).tocoo()
        I.col = I.col[col_order]
        new_M = new_M.dot(I)
    return new_M
#%timeit permute_sparse_matrix(A,perm,perm)

In [10]:
#example assemble
Np=7   #number of pipes
Nj=5  #number of junctions (nodes)
Nr=1
pipe=[0,1,2,3,4,5,6]  #number of pipes are consecutive, k=0:Np
i=[0,1,2,3,3,3,0]     #number of start node for each pipe, 0<=i<Nj
j=[1,5,1,1,2,4,5]     #number of end node for each pipe, 0<=j<Nj
b=[1.3,0.5,1.5,2.7,1.5,2.2,3.1]   #value for each pipe, Np elements
node=[0,1,2,3,4,5]   #number of nodes are consecutive, 0:Nj
F=np.zeros(Nj+Nr+2)   #intializing F, size is equal to number of nodes

nn=Nj+Nr
for k in range(Np):   #iterate over pipes
    F[i[k]]=F[i[k]]+b[k]   #add value of b if pipe k is at the start of the node
    F[j[k]]=F[j[k]]-b[k]   #substract b if pipe k is at the end of the node

F

array([ 4.4, -5. ,  0. ,  6.4, -2.2, -3.6,  0. ,  0. ])